In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import time, os, sys, pickle
import numpy as np
from scripts.utils import set_seed
from scripts.ds_class import *
from scripts.homeos import *
from scripts.plotting import *
from scripts.fit_motif import *
from scripts.time_series import *
from scripts.ra import *

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import MaxNLocator
import seaborn as sns

set_seed(313)
from pathlib import Path
exp_dir = Path('experiments')
data_dir = exp_dir / 'all_targets'
save_dir = data_dir / 'motif_fits'

In [ ]:
save_dir.mkdir(parents=True, exist_ok=True)
npy_files = list(data_dir.glob('*.npy'))


for file in npy_files:
    target_name = file.name.removesuffix('.npy')

    target_trajs = np.load(file)
    print(target_name, 'dim = ', target_trajs.shape[2])
    file_ = data_dir / f'{target_name}.npy'



bz_reaction dim =  2
lienard_sigmoid dim =  2
lorenz_off_313 dim =  3
lorenz_on_313 dim =  3
noisy_vdp dim =  2
ra dim =  2
ra_nstd0.25 dim =  2
selkov dim =  2
vdp dim =  2


In [3]:
def run_on_target(target_name, save_dir, ds_motif = 'ring', analytic = True, maxT = 5,
                    alpha_init = None, velocity_init = None, vf_on_ring_enabled = False, #if analytic then not used
                    homeo_type = 'node', layer_sizes = 1*[64], 
                    train_ratio = 0.8, training_pairs = False, 
                    lr = 0.01, num_epochs = 1000, 
                    random_seed = 313):
    save_dir = os.path.join(save_dir, ds_motif)
    os.makedirs(save_dir, exist_ok=True)
    set_seed(random_seed)
     
    trajectories_target = np.load(data_dir / f'{target_name}.npy')
    dim = trajectories_target.shape[2]
    tsteps = trajectories_target.shape[1]
    B = trajectories_target.shape[0]

    dt = maxT / tsteps
    time_span = torch.tensor([0.0, maxT])
    
    if training_pairs:
        time_span = torch.tensor([0.0, dt])
    ds_params = {'ds_motif': ds_motif, 'dim': dim, 'dt': dt, 'time_span': time_span, 'analytic': analytic, 'vf_on_ring_enabled': vf_on_ring_enabled, 'alpha_init': alpha_init, 'velocity_init': velocity_init}
    init_type = None
    homeo_params = {'homeo_type': homeo_type, 'dim': dim, 'layer_sizes': layer_sizes, 'activation': nn.ReLU, 'init_type': init_type}
    annealing_params = {'dynamic': False, 'initial_std': .0, 'final_std': 0.}
    training_params = {'lr': lr, 'num_epochs': num_epochs, 'annealing_params': annealing_params, 'early_stopping_patience': 1000, "batch_size": 32, 'use_inverse_formulation': True}
    all_parameters = { 'homeo_params': homeo_params, 'training_params': training_params, "ds_params": ds_params}
    with open(f"{save_dir}/parameters_{target_name}.pkl", "wb") as f:
        pickle.dump(all_parameters, f)

    trajectories_target = torch.tensor(trajectories_target, dtype=torch.float32).to(device)
    trajectories_target_full, trajectories_target, mean, std = normalize_scale_pair(trajectories_target, False)

    n_train = int(train_ratio * B)
    n_test = B - n_train
    train_set, test_set = random_split(trajectories_target_full, [n_train, n_test])
    trajectories_target_train = trajectories_target[train_set.indices]
    trajectories_target_test = trajectories_target[test_set.indices]

    #train homeo_ds_net
    homeo = build_homeomorphism(homeo_params)
    source_system_ra = build_ds_motif(**ds_params)
    homeo_ds_net = Homeo_DS_Net(homeo, source_system_ra)
    homeo_ds_net, losses, grad_norms = train_homeo_ds_net_batched(homeo_ds_net=homeo_ds_net, trajectories_target=trajectories_target_train, **training_params)
    homeo_ds_net.eval()
    
    #test
    _, _, training_loss = test_single_homeo_ds_net(homeo_ds_net=homeo_ds_net, trajectories_target=trajectories_target_train)
    _, _, test_loss = test_single_homeo_ds_net(homeo_ds_net=homeo_ds_net, trajectories_target=trajectories_target_test)
    traj_src_np, traj_trans_np, _ = test_single_homeo_ds_net(homeo_ds_net=homeo_ds_net, trajectories_target=trajectories_target)

    inv_man = homeo_ds_net.invariant_manifold(100).detach().numpy()
    jac_norm_frobenius = jacobian_norm_over_batch(homeo_ds_net.homeo_network, trajectories_target.reshape(-1,dim), norm_type='fro').detach().numpy()
    jac_norm_spectral = jacobian_norm_over_batch(homeo_ds_net.homeo_network, trajectories_target.reshape(-1,dim), norm_type='spectral').detach().numpy()

    np.savez(
    f"{save_dir}/results_{target_name}.npz",
    jac_fro=jac_norm_frobenius,
    jac_spec=jac_norm_spectral,
    training_loss=training_loss,
    test_loss=test_loss,
    losses=np.array(losses),  
    grad_norms=np.array(grad_norms),
    inv_man=inv_man
)

    save_homeo_ds_net(homeo_ds_net, f"{save_dir}/homeo_{target_name}.pth")
    np.save(f"{save_dir}/traj_motif_transformed_{target_name}.npy", traj_trans_np) 
    np.save(f"{save_dir}/traj_motif_source_{target_name}.npy", traj_src_np) 

# 0D: LDS
# 1D: Bounded line attractor (bla)
# 2D motifs: 
- Ring attractor
- Limit cycle
- Two line attractors (bibla)
# 3D:
- Sphere
- Torus
-- with LC+RA
-- with RAs
- Cylinder LC+BLA

# 2D targets:
- Van der Pol (+noisy)
- Lienard
- BZ reaction
- Selkov
- RA (+noisy) #(+perturbed)

In [5]:
motifs_2d = ['lds', 'lc', 'ring', 'bla', 'bistable']

In [ ]:
file = npy_files[0]
target_name = file.name.removesuffix('.npy')
print("Starting training motifs on system", target_name)
for motif in motifs_2d:
    print('training motif', motif)
    run_on_target(target_name, save_dir=save_dir, ds_motif=motif, analytic=True)

Starting training motifs on system bz_reaction
training motif lds
Epoch 0, log(Loss)= 0.0203
Epoch 10, log(Loss)= -0.0249
Epoch 20, log(Loss)= -0.0938


# 3D:
- Lorenz LC

In [ ]:
run_on_target("lorenz_on_313", save_dir=save_dir, ds_motif='sphere', analytic=True)

Epoch 0, log(Loss)= -0.0317
Final log(Loss)= -0.0317, Total training time: 0.17 sec, Avg epoch time: 0.1676 sec
Computing Jacobian norms...
Computing Jacobian norms...

Model saved to experiments\all_targets\motif_fits\sphere/homeo_lorenz_on_313.pth
